In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
apple = cv2.imread('data/apple.jpg')
orange = cv2.imread('data/orange.jpg')

In [3]:
print(f"{apple.shape}")
print(f"{orange.shape}")

(512, 512, 3)
(512, 512, 3)


In [4]:
apple

array([[[ 91, 126, 122],
        [ 82, 119, 115],
        [ 87, 124, 120],
        ...,
        [103, 168, 153],
        [103, 168, 153],
        [105, 170, 155]],

       [[ 87, 122, 118],
        [ 77, 114, 110],
        [ 80, 117, 113],
        ...,
        [114, 181, 166],
        [116, 183, 168],
        [118, 185, 170]],

       [[ 82, 117, 113],
        [ 74, 111, 107],
        [ 78, 115, 111],
        ...,
        [108, 178, 161],
        [109, 180, 163],
        [111, 182, 165]],

       ...,

       [[113, 157, 116],
        [115, 160, 117],
        [120, 163, 120],
        ...,
        [109, 139,  98],
        [109, 141, 100],
        [111, 144, 100]],

       [[107, 151, 112],
        [109, 153, 112],
        [112, 155, 112],
        ...,
        [112, 143,  98],
        [111, 144,  99],
        [111, 144,  99]],

       [[113, 157, 118],
        [113, 157, 116],
        [117, 157, 116],
        ...,
        [125, 156, 111],
        [120, 154, 107],
        [118, 152, 105]]

In [5]:
orange

array([[[104, 174, 137],
        [105, 178, 140],
        [108, 181, 143],
        ...,
        [255, 254, 243],
        [253, 252, 242],
        [251, 250, 240]],

       [[ 96, 166, 129],
        [ 99, 169, 132],
        [100, 173, 135],
        ...,
        [252, 255, 240],
        [247, 251, 239],
        [244, 248, 236]],

       [[103, 172, 135],
        [103, 173, 136],
        [104, 177, 139],
        ...,
        [191, 203, 185],
        [188, 201, 185],
        [186, 199, 183]],

       ...,

       [[105, 107, 117],
        [105, 107, 117],
        [100, 105, 114],
        ...,
        [ 70,  68,  68],
        [ 72,  67,  68],
        [ 70,  68,  68]],

       [[203, 209, 222],
        [203, 209, 222],
        [198, 206, 219],
        ...,
        [ 72,  67,  68],
        [ 72,  66,  67],
        [ 73,  68,  69]],

       [[231, 238, 253],
        [232, 239, 254],
        [231, 240, 254],
        ...,
        [ 71,  65,  66],
        [ 71,  65,  66],
        [ 74,  68,  69]]

In [6]:
apple_orange = np.hstack((apple[:,:256], orange[:,256:]))

In [7]:
apple_orange

array([[[ 91, 126, 122],
        [ 82, 119, 115],
        [ 87, 124, 120],
        ...,
        [255, 254, 243],
        [253, 252, 242],
        [251, 250, 240]],

       [[ 87, 122, 118],
        [ 77, 114, 110],
        [ 80, 117, 113],
        ...,
        [252, 255, 240],
        [247, 251, 239],
        [244, 248, 236]],

       [[ 82, 117, 113],
        [ 74, 111, 107],
        [ 78, 115, 111],
        ...,
        [191, 203, 185],
        [188, 201, 185],
        [186, 199, 183]],

       ...,

       [[113, 157, 116],
        [115, 160, 117],
        [120, 163, 120],
        ...,
        [ 70,  68,  68],
        [ 72,  67,  68],
        [ 70,  68,  68]],

       [[107, 151, 112],
        [109, 153, 112],
        [112, 155, 112],
        ...,
        [ 72,  67,  68],
        [ 72,  66,  67],
        [ 73,  68,  69]],

       [[113, 157, 118],
        [113, 157, 116],
        [117, 157, 116],
        ...,
        [ 71,  65,  66],
        [ 71,  65,  66],
        [ 74,  68,  69]]

In [ ]:
cv2.imshow("Apple", apple)
cv2.imshow("Orange", orange)
cv2.imshow("AppleOrange", apple_orange)

if cv2.waitKey(0) & 0xFF == ord('q'):
    cv2.destroyAllWindows()

In [10]:
#generate Gaussian pyramid for apple
apple_copy = apple.copy()
gp_apple = [apple_copy]
for i in range(6):
    apple_copy = cv2.pyrDown(apple_copy)
    gp_apple.append(apple_copy)
    
    
#generate Gaussian pyramid for orange
orange_copy = orange.copy()
gp_orange = [orange_copy]
for i in range(6):
    orange_copy = cv2.pyrDown(orange_copy)
    gp_orange.append(orange_copy)

In [11]:
#generate Laplacian pyramid for apple
apple_copy = gp_apple[5]
lp_apple = [apple_copy]
for i in range(5,0,-1):
    gaussian_extended = cv2.pyrUp(gp_apple[i])
    laplacian = cv2.subtract(gp_apple[i-1],gaussian_extended)
    lp_apple.append(laplacian)
    
#generate Laplacian pyramid for orange
orange_copy = gp_orange[5]
lp_orange = [orange_copy]
for i in range(5,0,-1):
    gaussian_extended = cv2.pyrUp(gp_orange[i])
    laplacian = cv2.subtract(gp_orange[i-1],gaussian_extended)
    lp_orange.append(laplacian)
        

In [12]:
#Now add left and right halves of images in each level
apple_orange_pyramid = []
n = 0
for apple_lap, orange_lap in zip(lp_apple, lp_orange):
    n+=1
    cols, rows, channels = apple_lap.shape
    laplacian = np.hstack((apple_lap[:,0:int(cols/2)], 
                         orange_lap[:, int(cols/2):]))
    apple_orange_pyramid.append(laplacian)

    
#new reconstruct
apple_orange_reconstruct = apple_orange_pyramid[0]
for i in range(1,6):
    apple_orange_reconstruct = cv2.pyrUp(apple_orange_reconstruct)
    apple_orange_reconstuct = cv2.add(apple_orange_pyramid[i], apple_orange_reconstruct)
    

In [13]:
cv2.imshow("Apple", apple)
cv2.imshow("Orange", orange)
cv2.imshow("Apple Orange", apple_orange)
cv2.imshow("Apple Orange Reconstructed", apple_orange_reconstruct)

if cv2.waitKey(0) & 0xFF == ord('q'):
    cv2.destroyAllWindows()